# Table of Contents

### 1. [Setup](#Setup)


# Setup

- imports
- mkdirs
- constants
- variables

## Imports

In [35]:
import os
import csv
import time
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


In [2]:
# Try to import parquet engines
try:
    import pyarrow
    PARQUET_ENGINE = 'pyarrow'
except ImportError:
    try:
        import fastparquet
        PARQUET_ENGINE = 'fastparquet'
    except ImportError:
        print("Warning: No parquet engine found. Installing pyarrow recommended.")
        print("Run: pip install pyarrow")
        PARQUET_ENGINE = None

## Mkdirs

In [ ]:
# os.makedirs('data/processed', exist_ok=True)

## Consts and Vars

In [50]:
PARAMS = [
    # 'AvgSurfT_inst',
    # 'CanopInt_inst',
    # 'LWdown_f_tavg',
    # 'Psurf_f_inst',
    # 'Qair_f_inst',
    # 'SnowDepth_inst',
    # 'SWdown_f_tavg',
    # 'Tair_f_inst',
    # 'TVeg_tavg',
    # 'Wind_f_inst',
    'Rainf_tavg'
]  

AGGREGATION = {
    'Rainf_tavg': 'sum',        # Rain accumulates over time
    'SnowDepth_inst': 'sum',    # Snow accumulation over time  
    'CanopInt_inst': 'sum',     # Water accumulation over time
    'Tair_f_inst': 'mean',      # Daily average temperature
    'AvgSurfT_inst': 'mean',    # Daily average surface temperature
    'Psurf_f_inst': 'mean',     # Daily average pressure
    'Qair_f_inst': 'mean',      # Daily average humidity
    'Wind_f_inst': 'mean',      # Daily average wind speed
    'LWdown_f_tavg': 'mean',    # Daily average longwave radiation
    'SWdown_f_tavg': 'mean',    # Daily average shortwave radiation
    'TVeg_tavg': 'mean'         # Daily average transpiration
}

DATES = [
    '2024_March',
    '2024_April',
    # '2024_May',
    # '2024_June',
    # '2024_July',
    # '2024_Aug',
    # '2024_Sept',
    # '2024_Oct',
    # '2024_Nov',
    # '2024_Dec',
    # '2025_Jan',
    # '2025_Feb',
]

# ReUsable Functions

## Global Functions

In [4]:
def read_raw_csv(filename: str) -> pd.DataFrame:
    """Read a CSV file and return a pandas DataFrame."""
    file_path = Path('data/raw') / filename
    return pd.read_csv(file_path)

In [27]:
def read_csv_to_numpy(filename: str, subfolder: str) -> np.ndarray:
    filename = f"{filename}.csv"
    file_path = os.path.join('data', subfolder, filename)

    data_as_list = []

    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')

        next(csv_reader)  # Skip header
        
        for row in csv_reader:
            data_as_list.append([float(val) for val in row])
    
    data = np.array(data_as_list)
    return data

## Complex Functions

In [52]:
def aggregate_3hourly_to_daily_pandas(arr, parameter):
    # Convert array to pandas DataFrame
    df = pd.DataFrame(arr, columns=['year', 'month', 'day', 'hour', 'longitude', 'latitude', 'value'])
    
    # Get the aggregation method for this parameter
    agg_method = AGGREGATION.get(parameter, 'mean')  # Default to 'mean' if not found
    
    # Aggregate to daily using the appropriate method
    daily_df = df.groupby(['longitude', 'latitude', 'year', 'month', 'day'])['value'].agg(agg_method).reset_index()
    
    # Convert back to numpy array
    daily_arr = daily_df.values
    
    return daily_arr

# Raw Data Proccess

- Loop through and read monthly parameter CSVs
- Run basic checks: missing data
- Select data types so can be held as numpy array
- Aggregate to daily
- Concat monthlys into a single array
- Sort by long, lat, year, month, day
- Save as parqets in aggregated folder

In [51]:
for parameter in PARAMS:
    print(f'Parameter: {parameter}')

    monthly = []

    for date in DATES:
        print(f"\n===\n")

        # File to be processed:
        print(f"date: {date}")
        filename = f"{parameter}_data_{date}"
        print(filename + '.csv')

        # Load CSV to numpy array:
        try:
            arr = read_csv_to_numpy(filename, 'raw')
        except Exception as e:
            print(f"Error: {e}")
        
        # Basic statis/checks:
        print(f"Total rows: {arr.shape[0]}")
        print(f"Missing values in final column: {np.isnan(arr[:, -1]).sum()}")
        print(f"Missing percentage: {np.isnan(arr[:, -1]).sum() / arr.shape[0] * 100:.2f}%")

        print(f"Range: {arr[:, -1].min():.2f} to {arr[:, -1].max():.2f}")
        print(f"Average: {arr[:, -1].mean():.2f}")

        print(f"Longitude range: {arr[:, 4].min():.2f} to {arr[:, 4].max():.2f}")
        print(f"Latitude range: {arr[:, 5].min():.2f} to {arr[:, 5].max():.2f}")

        # Aggregation from 3hourly to daily:
        daily_arr_numpy = aggregate_3hourly_to_daily_pandas(arr, parameter)
        print(f"Original 3-hourly data: {arr.shape}")
        print(f"Aggregated daily data: {daily_arr_numpy.shape}")

        print(f"\n===\n")

    # print(f'monthly: {monthly}')

Parameter: Rainf_tavg

===

date: 2024_March
Rainf_tavg_data_2024_March.csv
Total rows: 3759432
Missing values in final column: 0
Missing percentage: 0.00%
Range: 0.00 to 0.01
Average: 0.00
Longitude range: -179.50 to 179.50
Latitude range: -54.50 to 83.50
Rainf_tavg sum
Original 3-hourly data: (3759432, 7)
Aggregated daily data: (469929, 6)

===


===

date: 2024_April
Rainf_tavg_data_2024_April.csv
Total rows: 3638160
Missing values in final column: 0
Missing percentage: 0.00%
Range: 0.00 to 0.01
Average: 0.00
Longitude range: -179.50 to 179.50
Latitude range: -54.50 to 83.50
Rainf_tavg sum
Original 3-hourly data: (3638160, 7)
Aggregated daily data: (454770, 6)

===

